## Section Headings Curation of El Salvador Policies

In this notebook there are a series of dictionaries and methods to curate section headings of El Salvador policies. Policies from El Salvador have a rather definite structure, so that the law text is organized under section headings. There are two kinds of sections, the ones that are general and that can be often found in many policies, and the ones which are specific. The sections headings which are more general often come with a whole range of name variants which makes the task of machine recognition difficult.

The goal of this notebook is to group all pretreatment methods that would harmonize sections heading to make the further processing machine friendly.

In [1]:
from pathlib import Path
import re, boto3, json, string, operator
import numpy as np

### Dictionaries of particular vocabularies to help in the curation of section headings

In [23]:
# Most policies come with the final signatures. This is a piece of text that we want to be able to recognize. To make the
# detection of signatures easier, this dictionary contain the most common terms that can be found in these lines of text.
official_positions = {"ALCALDE" : 0,
"Alcalde" : 0,
"MINISTRA" : 0,
"Ministra" : 0,
"MINISTRO" : 0,
"Ministro" : 0,
"PRESIDENTA" : 0,
"Presidenta" : 0,
"PRESIDENTE" : 0,
"Presidente" : 0,
"REGIDOR" : 0,
"Regidor"  : 0,
"REGIDORA" : 0,
"regidora" : 0,
"SECRETARIA" : 0,
"Secretaria" : 0,
"SECRETARIO" : 0,
"Secretario" : 0,
"SINDICA" : 0,
"Sindica" : 0,
"SINDICO" : 0,
"Sindico" : 0,
"VICEPRESIDENTA" : 0,
"Vicepresidenta" : 0,
"VICEPRESIDENTE" : 0,
"Vicepresidente" : 0
}
# This dictionary contains some correspondences among different text headings. This is under development and needs further
# improvement.The idea is to merge in a single name all the headings that point to the same conceptual concept. For example,
# "Definiciones" is a heading that can come alone or together with other terms so it can appear as "Definiciones básicas" or
# "Definiciones generales". With the dictionary we can fetch all headings that contain the word "Definiciones" and change the
# heading to "Definiciones".
merges = {
    "CONCEPTOS" : "DEFINICIONES",
    "DEFINICIONES" : "DEFINICIONES",
    "DISPOSICIONES FINALES" : "DISPOSICIONES GENERALES",
    "DISPOSICIONES GENERALES" : "DISPOSICIONES GENERALES",
    "DISPOSICIONES PRELIMINARES" : "DISPOSICIONES PRELIMINARES",
    "DISPOSICIONES REGULADORAS" : "DISPOSICIONES ESPECIALES",
    "DISPOSICIONES RELATIVAS" : "DISPOSICIONES ESPECIALES",
    "DISPOSICIONES ESPECIALES" : "DISPOSICIONES ESPECIALES",
    "DISPOSICIONES TRANSITORIAS" : "DISPOSICIONES GENERALES",
    "INFRACCIONES" : "INFRACCIONES",
    "INFRACCION ES" : "INFRACCIONES",
    "OBJETIVO" : "OBJETO",
    "OBJETO" : "OBJETO",
    "OBLIGACIONES" : "OBLIGACIONES Y PROHIBICIONES",
    "OBLIGACIONE" : "OBLIGACIONES Y PROHIBICIONES",
    "OBLIGACION" : "OBLIGACIONES Y PROHIBICIONES",
    "OBLIGATORIEDAD" : "OBLIGACIONES Y PROHIBICIONES",
    "POR TANTO" : "POR TANTO",
    "POR LO TANTO" : "POR TANTO",
    "PROHIBICIONES" : "OBLIGACIONES Y PROHIBICIONES",
    "PROHIBICION" : "OBLIGACIONES Y PROHIBICIONES"
}
# Eventhough the general gramar rule in Spanish is not to accent uppercase, there are many cases where a word in a heding might
# appear accented. This is a dictionary to armonize all headings without accents. The list is rather comprehensive, but there is
# still room for improvement.
# If we find some bug beyond simple misspelling which will be solved by spell checker, we can include it here. The example is in
# the first row with "ACTIVIDADESUSOS" which was found several times in headings.
bugs = {"ACTIVIDADESUSOS" : "ACTIVIDADES DE USOS"}

### Connection to the AWS S3 bucket
To effectively run this cell you need Omdena's credentials. Please keep them local and do not sync them in GitHub repos nor cloud drives. Before doing anything with this json file, please think of security!!

In [3]:
json_folder = Path("C:/Users/user/Google Drive/Els_meus_documents/projectes/CompetitiveIntelligence/WRI/Notebooks/credentials/")
filename = "Omdena_key.json"
file = json_folder / filename

with open(file, 'r') as f:
    key_dict = json.load(f) 

for key in key_dict:
    KEY = key
    SECRET = key_dict[key]

s3 = boto3.resource(
    service_name = 's3',
    region_name = 'us-east-2',
    aws_access_key_id = KEY,
    aws_secret_access_key = SECRET
)

### Functions and regular expressions

In [15]:

# Function to calculate the uppercase ratio in a string. It is used to detect section headings
def uppercase_ratio(string):
    return(len(re.findall(r'[A-Z]',string))/len(string))

# Regular expression to clear html tags (here is basically to remove the page tags)
cleanr = re.compile(r'<.*?>')
# Te function to clear html tags
def clean_html_tags(string):
  return cleanr.sub('', string)

# Function to remove the last lines of a document, the ones that contain the signatures of the officials. It depends on the
# dictionary "official_positions"
def remove_signatures(line):
    signature = False
    for key in official_positions:
        if key in line:
            signature = True
            break
    return signature

# Function to change accented words by non-accented counterparts. It depends on the dictionary "accent_marks_bugs" 
accents_out = re.compile(r'[áéíóúÁÉÍÓÚ]')
accents_dict = {"á":"a","é":"e","í":"i","ó":"o","ú":"u","Á":"A","É":"E","Í":"I","Ó":"O","Ú":"U"}
def remove_accents(string):
    for accent in accents_out.findall(string):
        string = string.replace(accent, accents_dict[accent])
    return string

# Function to merge headlines expressing the same concept in different words. It depends on the dictionary "merges"
def merge_concepts(line):
    for key in merges:
        if key in line:
            line = merges[key]
            break
    return line

def clean_bugs(line):
    for key in bugs:
        if key in line:
            line = line.replace(key, bugs[key])
    return line

# Function to add items to the dictionary with duplicate removal
def add_to_dict(string, dictionary, dupl_dict):
    if string in dupl_dict or string == None:
        pass
    else:
        dupl_dict[string] = 0
        if string in dictionary:
            dictionary[string] = dictionary[string] + 1
        else:
            dictionary[string] = 1
    return dictionary

# Regular expression to clear punctuation from a string
clean_punct = re.compile('[%s]' % re.escape(string.punctuation))
# Regular expression to clear words that introduce unnecessary variability to headings. Some regex still not work 100% we need
# to improve them.
clean_capitulo = re.compile(r'(APARTADO \S*)|(APARTADO\s)|(\bART. \S*)|(\bART. \d*)|(\bArt. \S*)|(Capítulo \S*)|(CAPITULO \S*)|(CAPITULO\S*)|(CAPÍTULO \S*)|(CAPITULÓ \S*)|(CAPITULOS \S*)|(CAPITUO \S*)|(CATEGORIA\b)|(CATEGORÍA\b)|(SUBCATEGORIA\b)|(SUBCATEGORÍA\b)|(TITULO\s\S*)|(TÍTULO\s\S*)')
# Function to remove 1. unwanted words; 2. punctuation; 3. leading white spaces
def clean_headings(string):
    string = clean_capitulo.sub('', string)
    clean_string = clean_punct.sub('', string).rstrip().lstrip()
    if clean_string != "":
        return clean_string
    else:
        pass


### Pipeline to process files from S3 bucket
By executing this cell you will go through all policies in El Salvador and process section headings that will be saved in a dictionary. This should be merged with the notebook that builds up the final json files out of plain txt files.

In [20]:
folder = "text-extraction/"
filename = "00a55afe4f55256567397a68df5d7f97e642480b" # This is only if you want to test on a single file
bag_of_words = {}
sentences = []

i = 0
for obj in s3.Bucket('wri-latin-talent').objects.all().filter(Prefix='text-extraction'):
    if folder in obj.key and obj.key.replace(folder, "") != "": # and filename in obj.key # Un comment the previous string to run the code just in one sample document.
#         print(i, "**", obj.key)
        file = obj.get()['Body'].read().decode('utf-8')  #get the file from S3 and read the body content
        lines = file.split("\n") # Split by end of line and pipe lines into a list
        duplicates_dict = {} #Sometimes the same heading can be found more than once in a document. This will help on removing them
        for line in lines:
            if uppercase_ratio(clean_html_tags(line)) > 0.6 and len(line) > 6:
                if remove_signatures(line):
                    break
                else:
                    line = clean_html_tags(line)
                    line = remove_accents(line)
                    line = clean_bugs(line)
                    line = clean_headings(line)
                    if line == None:
                        continue
                    line = merge_concepts(line)
                    bag_of_words = add_to_dict(line, bag_of_words, duplicates_dict)
            else:
                sentences.append(line)
#                 print("--", line)
#             s3.Object('wri-latin-talent', key).put(Body = content)#This will save all the contents in the string variable "content" into a txt file in the Pre-processed folder
        i += 1

#### Short summary

In [ ]:
print("After preprocessing there are {} different headings in El Salvador policies".format(len(bag_of_words)))
print("{} documents have been processed".format(i))
print("There are {} lines of text as sentences".format(len(sentences)))

#### Dictionary items sorted by occurrence

In [21]:
dict( sorted(bag_of_words.items(), key=operator.itemgetter(1),reverse=True))

{'CONSIDERANDO': 316,
 'POR TANTO': 279,
 'DISPOSICIONES GENERALES': 156,
 'OBJETO': 132,
 'OBLIGACIONES Y PROHIBICIONES': 128,
 'INFRACCIONES': 113,
 'DEFINICIONES': 103,
 'DECRETA': 100,
 'AMBITO DE APLICACION': 52,
 'DISPOSICIONES PRELIMINARES': 51,
 'DE LAS SANCIONES': 28,
 'VIGENCIA': 28,
 'DE LOS PERMISOS': 28,
 'AUTORIDAD COMPETENTE': 27,
 'EL ORGANO EJECUTIVO EN EL RAMO DE MEDIO AMBIENTE Y RECURSOS NATURALES': 26,
 'MEDIDAS PREVENTIVAS': 25,
 'LA ASAMBLEA LEGISLATIVA DE LA REPUBLICA DE EL SALVADOR': 23,
 'FORMULACION APROBACION Y CONTENIDO DE LOS PLANES PARCIALES': 18,
 'DE LAS PARCELACIONES': 18,
 'PROCEDIMIENTO': 18,
 'DE LOS USOS DEL SUELO': 17,
 'DEL PROCEDIMIENTO': 17,
 'COMPETENCIA': 16,
 'DE LAS ZONAS DE PROTECCION': 16,
 'COORDINACION INTERINSTITUCIONAL': 14,
 'APLICACION SUPLETORIA DE OTRAS NORMAS': 14,
 'CLASIFICACION Y REGIMEN URBANISTICO DEL SUELO': 14,
 'DE LOS TRAMITES Y SERVICIOS': 14,
 'DE LA RESPONSABILIDAD DE PROFESIONALES EMPRESAS PROPIETARIOS E INSTITUCIONES

#### Dictionary items sorted by heading text

In [22]:
for k in sorted(bag_of_words):
    print(k, ":", bag_of_words[k])

007 IPOL 39 SENNA A SOUL On SOU TE ESA DO UORONAIIOS E ClEO SEAS IO0U SEXO : 1
090115 JUEQIN OJUBILUEYODSADAd : 1
1 DE LAS SANCIONES AUTORIDADES COMPETENTES PROCEDIMIENTOS Y RECURSOS 1 : 1
1 LIMITE CAN Y ESTATAL : 1
11503 CONSIDERADOS EN PTU : 1
2   FUNCIONES Y ATRIBUCIONES DE LA MUNICIPALIDAD : 1
2 ORDENANZA REGULADORA DE USO DE SUELOS DEL y e MUNICIPIO DE SANTA TECLA a : 1
7 ADAPTACION AL CAMBIO CLIMATICO 7 : 1
7 DE LA ADAPTACION AL CAMBIO CLIMATICO 7 : 1
7 NIVELES O PISOS EN ADELANTE 40 TASAS : 3
A A o a ESPERAN E REINA AA PERA Ap ARA A EARL AE AO AOL lus cord 1016 ¡BA BOO IR URIA A BORIS CURS CANINO AA UA RUANO PATAEIUAS AA : 1
A ASA IAN : 1
A TE AAA : 1
AA PO ER : 1
ACCESO A LA PESCA Y LA ACUICULTURA : 1
ACCESO PROTECCION Y APROVECHAMIENTO DE LA DIVERSIDAD BIOLOGICA : 1
ACCION EJECUTIVA : 1
ACCIONES CONJUNTAS E INSTITUCIONALES : 1
ACCIONES DE POLITICA PUBLICA : 1
ACCIONES DE PREVENCION Y PROTECCION : 1
ACCIONES EN CASO DE EMERGENCIA : 10
ACCIONES Y MEDIDAS ESPECIALES DE CONSERVACI

DE LAS SERVIDUMBRES : 1
DE LAS SOLICITUDES : 3
DE LAS TASAS : 1
DE LAS TASAS MUNICIPALES POR SERVICIO DE PODA Y TALA DE EJEMPLARES ARBOREOS : 3
DE LAS TASAS Y SERVICIOS MUNICIPALES POR MANTENIMIENTO DE ARBOLES : 1
DE LAS TRANSGRESIONES SANCIONES Y RECURSOS : 1
DE LAS VENTAS Y EL VENDEDOR : 1
DE LAS ZONAS DE PROTECCION : 16
DE LAS ZONAS DE PROTECCION DE RECARGA ACUIFERA : 2
DE LAS ZONAS DE PROTECCION MUNICIPAL : 1
DE LAS ZONAS DE PROTECCION Y CONSERVACION ESPECIAL : 4
DE LAS ZONAS DE RIESGO Y MAXIMA PROTECCION : 1
DE LAS ZONAS DE USO RESTRINGIDO : 1
DE LAS ZONAS PROTEGIDAS : 1
DE LOS ANIMALES SALVAJES Y DOMESTICOS : 1
DE LOS ANIMALES Y SUS DEPOSICIONES : 2
DE LOS ARANCELES : 8
DE LOS ARBOLES Y BOSQUES : 1
DE LOS BOSQUES : 1
DE LOS CONCEPTOS GENERALES : 4
DE LOS CRITERIOS PARA AUTORIZAR LA PODA : 1
DE LOS CRITERIOS PARA AUTORIZAR LA TALA : 1
DE LOS CRITERIOS PARA LA TALA DE EJEMPLARES ARBOREOS : 4
DE LOS DEBERES DE LAS PERSONAS : 7
DE LOS DEBERES DE PROTECCION : 2
DE LOS DEBERES DE PROTE

EDUCACION Y FORMACION AMBIENTAL : 1
EFECTOS DE LA TERMINACION DE CONCESIONES DE EXPLOTACION : 1
EFRAIN PONCE ZELAYANDIA JOSE MARIO GARCIA : 1
EJERCICIO ILEGAL DE ACTIVIDADES MINERAS : 1
EL ALMACENAMIENTO TEMPORAL DE LOS RESIDUOS : 1
EL CONCEJO MUNICIPAL : 1
EL CONCEJO MUNICIPAL DE ARAMBALA DEPARTAMENTO DE MORAZAN : 1
EL CONCEJO MUNICIPAL DE CANDELARIA DE LA FRONTERA DEPARTAMENTO DE SANTA ANA : 1
EL CONCEJO MUNICIPAL DE CHALATENANGO DEPARTAMENTO DE CHALATENANGO : 1
EL CONCEJO MUNICIPAL DE CHILTIUPAN DEPARTAMENTO DE LA LIBERTAD : 1
EL CONCEJO MUNICIPAL DE CINQUERA DEPARTAMENTO DE CABAÑAS : 1
EL CONCEJO MUNICIPAL DE CITALA DEPARTAMENTO DE CHALATENANGO : 1
EL CONCEJO MUNICIPAL DE CIUDAD BARRIOS DEPARTAMENTO DE SAN MIGUEL : 1
EL CONCEJO MUNICIPAL DE COLON DEPARTAMENTO DE LA LIBERTAD : 2
EL CONCEJO MUNICIPAL DE CONCEPCION DE ORIENTE DEPARTAMENTO DE LA UNION : 1
EL CONCEJO MUNICIPAL DE CONCHAGUA DEPARTAMENTO DE LA UNION : 1
EL CONCEJO MUNICIPAL DE CUISNAHUAT DEPARTAMENTO DE SONSONATE : 1
EL C

LUIS ENGELBERTO ALEJO SIGUENZA : 1
La siguiente ORDENANZA PARA LA CONSERVACION INCREMENTO Y APROVECHAMIENTO : 1
MADERAS BLANDAS : 1
MADERAS DURAS : 1
MADERAS PALMACEAS BLANDAS : 1
MANANTIAL : 3
MANEJO ADECUADO DE LAS AGUAS RESIDUALES : 3
MANEJO DE LOS SUELOS Y ECOSISTEMAS TERRESTRES : 1
MANEJO ESPECIAL CON MEDIDAS DE PROTECCION DE SUELOS : 1
MANEJO INTEGRAL DE DESECHOS SOLIDOS : 1
MANEJO SOSTENIBLE DE LOS BOSQUES AREAS NATURALES Y ZONAS DE AMORTIGUAMIENTO DEL MUNICIPIO : 2
MANEJO SOSTENIBLE DE LOS BOSQUES AREAS NATURALES Y ZONAS DE AMORTIGUAMIENTO DEL MUNICIPIO DE LOS BOSQUES : 1
MANEJO Y DISPOSICION DE LOS RESIDUOS SOLIDOS : 1
MANEJO Y DISPOSICION FINAL DE DESECHOS SOLIDOS : 1
MANEJO Y DISPOSICION FINAL DE DESECHOS SOLIDOS EN LA ZONA RURAL : 5
MANIFIESTO : 1
MANTENIMIENTO Y OPERACION DEL SISTEMA : 1
MANTOS ACUIFEROS : 1
MARCO INSTITUCIONAL : 1
MARCO LEGAL Y FACTORES PARA FALLO : 2
MARINAS EN EL MUNICIPIO DE ACAJUTLA : 1
MARINAS EN EL MUNICIPIO DE SAN LUIS TALPA : 1
MARIO ANTONIO OSORT

PROTECCION DE LA ATMOSFERA : 1
PROTECCION DE LA FAUNA EN VIDA SILVESTRE : 1
PROTECCION DE LA VIDA SILVESTRE : 7
PROTECCION DE LOS BOSQUES Y EL SUELO : 1
PROTECCION DE LOS RECURSOS FORESTALES : 2
PROTECCION DE LOS RECURSOS HIDRICOS : 5
PROTECCION DE LOS RECURSOS NATURALES : 1
PROTECCION DE LOS RECURSOS NATURALES Y DEL MEDIO AMBIENTE : 2
PROTECCION DE LOS RECURSOS NATURALES Y EL MEDIO AMBIENTE : 1
PROTECCION DE RECURSOS NATURALES : 1
PROTECCION DE RECURSOS NATURALES Y DEL MEDIO AMBIENTE : 1
PROTECCION DE RIOS QUEBRADAS Y LUGARES DE ALTO RIESGO : 1
PROTECCION DE ZONAS DE RECARGA : 1
PROTECCION DEL MEDIO AMBIENTE : 3
PROTECCION DEL MEDIO COSTEROMARINO : 1
PROTECCION DEL RECURSO AGUA ACTIVIDADES A CORREGIR : 1
PROTECCION DEL RECURSO BOSQUE : 1
PROTECCION DEL RECURSO HIDRICO : 3
PROTECCION DEL RECURSO HIDRICO Y FORESTAL : 1
PROTECCION DEL RECURSO SUELO ACTIVIDADES A CORREGIR : 1
PROTECCION DEL SUELO : 1
PROTECCION DEL SUELO POR SU CAPACIDAD DE USO : 1
PROTECCION Y CONSERVACION AL MEDIO AMBIE

#### Saving sentences as csv

In [ ]:
print(sentences[0:2])

In [ ]:
path = Path("C:/Users/user/Google Drive/Els_meus_documents (1)/projectes/CompetitiveIntelligence/WRI/Notebooks/Data/")
filename = "sentences.npy"
file = path / filename
np_sentences = np.array(sentences)
with open(file, 'wb') as f:
    np.save(f, np_sentences)

#### Pipeline to process one file from HD folder
This is a pipeline to process a test file in a local folder.

In [ ]:
data_folder = Path("../Documents_de_mostra/")
filename = "00a55afe4f55256567397a68df5d7f97e642480b.pdf.txt"
bag_of_words = {}

i = 0
file = data_folder / filename
with open(file, 'r', encoding = 'utf-8') as file:
    lines = file.readlines()
    duplicates_dict = {}
    for line in lines:
        line = clean_html_tags(line)
        if uppercase_ratio(line) > 0.6 and len(line) > 6:
            if remove_signatures(line):
                break
            else:
#                 print(line)
                line = remove_accents(line)
                clean_line = clean_headings(line)
                bag_of_words = add_to_dict(clean_line, bag_of_words, duplicates_dict)
#                 print(clean_line)
        i += 1
#     data = file.read().replace('\n', '')

In [ ]:
bag_of_words